In [1]:
from scipy.io import loadmat
mnist = loadmat("mnist-original")
mnist_data = mnist["data"].T
mnist_label = mnist["label"][0]

In [2]:
mnist_data = mnist_data.reshape(70000, 28, 28, 1).astype("float32")/255
x, y = mnist_data, mnist_label

In [3]:
from sklearn.model_selection import train_test_split
train_images, test_images, train_labels, test_labels = train_test_split(x,y, test_size=0.1)

In [4]:
from keras.utils import to_categorical


train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [5]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Conv2D(32,(3,3), activation = "relu", input_shape = (28, 28, 1)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3), activation = "relu"))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3), activation = "relu"))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation = "relu"))
model.add(layers.Dense(10,activation= 'softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 576)               0

In [6]:
model.compile(optimizer = "rmsprop",
             loss = "categorical_crossentropy", 
             metrics = ["accuracy"])

In [7]:
model.fit(train_images, 
         train_labels,
         epochs = 10,
         batch_size = 128)

Epoch 1/10
493/493 [==============================] - 28s 54ms/step - loss: 0.2325 - accuracy: 0.9280
Epoch 2/10
493/493 [==============================] - 27s 55ms/step - loss: 0.0547 - accuracy: 0.9832
Epoch 3/10
493/493 [==============================] - 29s 59ms/step - loss: 0.0375 - accuracy: 0.9881
Epoch 4/10
493/493 [==============================] - 30s 62ms/step - loss: 0.0289 - accuracy: 0.9908
Epoch 5/10
493/493 [==============================] - 31s 62ms/step - loss: 0.0220 - accuracy: 0.9933
Epoch 6/10
493/493 [==============================] - 32s 64ms/step - loss: 0.0179 - accuracy: 0.9942
Epoch 7/10
493/493 [==============================] - 34s 68ms/step - loss: 0.0140 - accuracy: 0.9954
Epoch 8/10
493/493 [==============================] - 35s 71ms/step - loss: 0.0111 - accuracy: 0.9964
Epoch 9/10
493/493 [==============================] - 34s 68ms/step - loss: 0.0097 - accuracy: 0.9968
Epoch 10/10
493/493 [==============================] - 33s 68ms/step - loss: 0.007

In [8]:
model.evaluate(test_images, test_labels)

219/219 [==============================] - 2s 7ms/step - loss: 0.0268 - accuracy: 0.9930


[0.026793967932462692, 0.9929999709129333]

In [9]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(df.drop(['target'],axis='columns'),df.target, test_size=0.1)

In [10]:
import cv2
from imutils import contours


image = cv2.imread('image1.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(gray,0,200,cv2.THRESH_OTSU + cv2.THRESH_BINARY)[1]

cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
cnts, _ = contours.sort_contours(cnts, method="left-to-right")
Padding = 2
Border = 10

ROI_number = 0
for c in cnts:
    area = cv2.contourArea(c)
    if area > 10:
        x,y,w,h = cv2.boundingRect(c)
        ROI = gray[y-Padding:y+h+Padding, x-Padding:x+w+Padding]
        ROI = cv2.resize(ROI, (28,28))
        ROI = cv2.copyMakeBorder(ROI, Border,Border,Border,Border, cv2.BORDER_CONSTANT, value=[0,0,0])
        ROI = cv2.resize(ROI, (28,28))
        cv2.imwrite('chars/ROI_{}.png'.format(ROI_number), ROI)
        cv2.rectangle(image, (x, y), (x + w, y + h), (36,255,12), 1)
        ROI_number += 1

In [11]:
ROI_number

5

In [12]:
import numpy as np
chars = np.ndarray(shape=(ROI_number, 28, 28, 1))
for i in range(ROI_number):
    image = cv2.imread('chars/ROI_{}.png'.format(i))
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = gray.reshape(28,28,1).astype("float32")/255
    chars[i] = gray

In [13]:
chars.shape

(5, 28, 28, 1)

In [14]:
output = model.predict(chars)

1/1 [==============================] - 0s 153ms/step


In [15]:
for i in range(len(output)):
    print(np.where(output[i] == max(output[i]))[0][0], max(output[i]))

6 0.5695939
4 0.8671299
4 0.98110306
5 0.86445713
4 0.9996743


In [16]:
import pandas as pd

In [17]:
alpha = pd.read_csv("digital_letters.csv")

In [18]:
alpha.columns

Index(['Unnamed: 0', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       ...
       '775', '776', '777', '778', '779', '780', '781', '782', '783', 'label'],
      dtype='object', length=786)

In [19]:
y = alpha["label"]
y = y.to_numpy()

In [20]:
x = alpha.drop(['Unnamed: 0', 'label'], axis=1)

x = x.to_numpy()
total = x.shape[0]

x = x.reshape(total, 28, 28, 1).astype("float32")/255

In [21]:
alphabet = [chr(i) for i in range(ord("A"), ord("Z")+1)]
def ndarraytoord(arr):
    newarr = np.ndarray(shape=arr.shape)
    for idx, val in np.ndenumerate(arr):
        newarr[idx] = ord(val)-65
    return newarr

In [22]:
y = ndarraytoord(y)

In [23]:
from sklearn.model_selection import train_test_split
train_images, test_images, train_labels, test_labels = train_test_split(x,y, test_size=0.1)

In [24]:
from keras.utils import to_categorical


train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [25]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Conv2D(32,(3,3), activation = "relu", input_shape = (28, 28, 1)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3), activation = "relu"))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3), activation = "relu"))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation = "relu"))
model.add(layers.Dense(26,activation= 'softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 flatten_1 (Flatten)         (None, 576)              

In [26]:
model.compile(optimizer = "rmsprop",
             loss = "categorical_crossentropy", 
             metrics = ["accuracy"])

In [27]:
model.fit(train_images, 
         train_labels,
         epochs = 10,
         batch_size = 64)

Epoch 1/10
185/185 [==============================] - 7s 31ms/step - loss: 1.8956 - accuracy: 0.4908
Epoch 2/10
185/185 [==============================] - 6s 30ms/step - loss: 1.1115 - accuracy: 0.7069
Epoch 3/10
185/185 [==============================] - 6s 31ms/step - loss: 0.8977 - accuracy: 0.7586
Epoch 4/10
185/185 [==============================] - 6s 32ms/step - loss: 0.7766 - accuracy: 0.7866
Epoch 5/10
185/185 [==============================] - 6s 33ms/step - loss: 0.6858 - accuracy: 0.8102
Epoch 6/10
185/185 [==============================] - 6s 34ms/step - loss: 0.6136 - accuracy: 0.8276
Epoch 7/10
185/185 [==============================] - 6s 35ms/step - loss: 0.5515 - accuracy: 0.8451
Epoch 8/10
185/185 [==============================] - 7s 35ms/step - loss: 0.4962 - accuracy: 0.8615
Epoch 9/10
185/185 [==============================] - 7s 37ms/step - loss: 0.4444 - accuracy: 0.8714
Epoch 10/10
185/185 [==============================] - 7s 38ms/step - loss: 0.3925 - accura

In [28]:
output = model.predict(chars)

1/1 [==============================] - 0s 95ms/step


In [29]:
for i in range(len(output)):
    print(alphabet[np.where(output[i] == max(output[i]))[0][0]],max(output[i]))

B 0.99954176
L 0.9993783
A 0.9740691
C 0.99345124
K 0.996772
